In [1]:
import re
from pdfminer.high_level import extract_pages, extract_text

In [2]:
i = 0
for page_layout in extract_pages('bills.pdf'):
    for element in page_layout:
        # text = extract_text(element)

        if i == 3:
            print(element)
    if i == 3:
        break
    i += 1

<LTTextBoxHorizontal(0) 32.932,676.021,90.467,685.626 'Small World\n'>
<LTTextBoxHorizontal(1) 32.932,658.099,51.616,665.646 'From\n'>
<LTTextBoxHorizontal(2) 32.932,635.826,141.236,646.803 'DELHIVERY LIMITED \n'>
<LTTextBoxHorizontal(3) 32.932,621.051,119.968,628.598 'I FLOOR, 82, SECTOR-44 \n'>
<LTTextBoxHorizontal(4) 32.932,606.643,104.188,614.190 'Gurgaon, HARYANA \n'>
<LTTextBoxHorizontal(5) 32.932,592.235,149.469,599.782 'Pin code: 122002, State Code: 06 \n'>
<LTTextBoxHorizontal(6) 32.932,577.827,127.515,585.374 'GSTIN : 06AAPCS9575E1ZR \n'>
<LTTextBoxHorizontal(7) 180.570,687.414,324.128,707.310 '  TAX INVOICE\n'>
<LTTextBoxHorizontal(8) 314.228,658.099,335.939,665.646 'Bill To\n'>
<LTTextBoxHorizontal(9) 314.228,620.732,497.617,646.803 'GLOBAL SERVICES , GLOBAL SERVI\nCES\n'>
<LTTextBoxHorizontal(10) 314.228,595.665,394.514,613.504 'IH-24A NIT FARIDABAD\nFaridabad, HARYANA \n'>
<LTTextBoxHorizontal(11) 314.228,581.257,430.765,588.804 'Pin code: 121005, State Code: 06 \n'>
<LTT

In [3]:
from pdfquery import PDFQuery

pdf = PDFQuery('bills.pdf')
pdf.load()

In [4]:
invoices = pdf.pq('LTTextBoxHorizontal:in_bbox("109.774,496.869,159.075,504.416")').text()

len(invoices.split(' '))

1

In [ ]:
%pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 23.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 28.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pdfminer.six
    Found existing installation: pdfminer.six 20231228
    Uninstalling pdfminer.six-20231228:
      Successfully uninstalled pdfminer.six-20231228

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pdfplumber
import pandas as pd

/var/folders/l4/ssmqg5cn6_53k7x8rn66lywm0000gn/T/ipykernel_28708/3904033354.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [6]:
i = 0
df = pd.DataFrame(columns=['Place of Supply', 'Invoice No.', 'Invoice Date', 'Markup Amount', 'Taxable Amount', 'IGST', 'CGST','SGST','Total Amount', 'Bill To'])

with pdfplumber.open('bills.pdf') as pdf:
    pages = pdf.pages
    for page in pages:
        text = page.extract_text()
        elements = text.split('\n')
        print(elements)
        place = ''
        invoice_no = ''
        invoice_date = ''
        markup = ''
        taxable_amount = ''
        igst = ''
        cgst = ''
        sgst = ''
        total_amount = ''
        bill_to = ''
        for element in elements:
            if element.startswith('Place of Supply'):
                print(' '.join(element.split(' ')[3:-2]))
                place = ' '.join(element.split(' ')[3:-2])
            if element.startswith('Invoice No.'):
                print(element.split(' ')[2])    
                invoice_no = element.split(' ')[2]
            if element.startswith('Invoice Date'):
                print(element.split(' ')[-1])    
                invoice_date = element.split(' ')[-1]    
            if element.startswith('Markup:'):
                print(element.split(' ')[-1])    
                markup = element.split(' ')[-1]
            if element.startswith('Taxable Amount:'):
                print(element.split(' ')[-1])    
                taxable_amount = element.split(' ')[-1]
            if element.startswith('IGST:'):
                print(element.split(' ')[-1])    
                igst = element.split(' ')[-1]    
            if element.startswith('CGST:'):
                print(element.split(' ')[-1])    
                cgst = element.split(' ')[-1]    
            if element.startswith('SGST:'):
                print(element.split(' ')[-1])    
                sgst = element.split(' ')[-1]    
            if element.startswith('Description of charges: As per agreed terms, Total Amount'):
                print(element.split(' ')[-1])
                total_amount = element.split(' ')[-1]    
        try:
            x = elements.index('From Bill To')
            x += 1
            bill_to = elements[x].split(' , ')[-1]
            print(bill_to)
        except:
            print('error')
        finally:
            pass
        # break
        row = [place, invoice_no, invoice_date, markup, taxable_amount, igst, cgst, sgst,total_amount, bill_to]
        if row != [''] * 10:
            df.loc[len(df)] = row
df.to_csv('bills.csv')
# df.to_excel('bills_excel.xlsx')

['TAX INVOICE', 'Small World', 'From Bill To', 'DELHIVERY LIMITED GLOBAL SERVICES , GLOBAL SERVI', 'CES', 'plot no. 338, Village Bhartal, Sector 26, South West Delhi', 'Delhi, DELHI IH-24A NIT FARIDABAD', 'Faridabad, HARYANA', 'Pin code: 110075, State Code: 07', 'Pin code: 121005, State Code: 06', 'GSTIN : 07AAPCS9575E1ZP', 'GSTIN : 06AEYPR6542L2ZJ', 'PAN : NA', 'Place of Supply MEGHALAYA Ship To', 'Invoice No. FODA24253300151 ROHIT,RAIKINS INTERIODESIGN PRIVATE LIMITED', 'RAIKINS INTERIODESIGN PRIVATE LIMITED Sangma', 'Invoice Date 08-JAN-2024', 'House, 3 Miles Uppar Shillong, Behind Bharat Petrol', 'SAC 9965 ium-793005 PH ROHIT 9864605371 RAVI 9871763252', 'Shillong, MEGHALAYA', 'IRN 9375c4d0c78932630ffdc9a551529', 'Pin code: 793005, State Code: 17', 'b2f08b21bdcbfc34f8cdec0434ab3', 'bbe025', 'Description Units Invoice Amount', 'Base Freight Charge', 'Base Charge: 21.5/Kg', 'Rs. 7588.46', 'Processing Charge: 100.00/LR', 'Fuel Charges: 20%', 'Insurance ROV: 0.05% Rs. 57.86', 'Other Ha

In [99]:
%pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 5.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
df.to_excel('delhivery_jan_bills.xlsx', sheet_name='bills', index=False, engine='xlsxwriter')